## "Target Profile of Players Who Contribute the Most Data"
### Introduction:

<p style="text-indent: 40px;"> A research group in the Department of Computer Science at UBC is studying how
people interact within a virtual environment. To collect data, researchers were able to gather a
wide variety of people and put them all into a Minecraft server in order to document their
actions. Data that was collected from this experiment included the players name, age, skill level,
gender, forum subscription status, hours played and previous experience in the game.
Additionally, researchers were also able to collect exact dates and time players logged into and
off the Minecraft server. The goal of this research server is to help identify various patterns of
engagement between the type of players and the hours spent in the game.

<p style="text-indent: 40px;">Although there are many ways to interpret and analyze patterns in the data, we have
chosen to target the question of: Which types of players are most likely to contribute to
large amounts of data on the Minecraft research server? More specifically, we aim to
examine whether certain player characteristics like age, total hours played, and typical session
lengths are associated with higher levels of activity inside of the Minecraft server.
Understanding the demographic of people who contribute the most with data collection helps us
better understand the traits, like age ranges, that could be strongly correlated with higher
engagement.

<p style="text-indent: 40px;">To address this question we used the players.csv and sessions.csv dataset to see the
demographic and behavior related to overall activity levels. The player.csv dataset, contains a
total of 196 observations and seven different variables documenting the players age, gender,
experience level, hours played and subscription status. The sessions.csv file contains 1535
observations and five variables that describe the start and end times of each gameplay session,
including timestamps. Using these two files together lets us see both who the players are and
how long they actually spent playing, which lets us figure out the type of plays which contribute
the most to the data.

### Methods & Results:

<center><b>Table 1.</b> Overview of Data with Number of Columns and Observations</center>

| Source File Name | Number of Columns | Total Number of Observations |
|:-:|:-:|:-:|
|players.csv| 7 | 196 |
|sessions.csv| 5 | 1535 | 


<center><b>Table 2.</b> Summary statistics for variables of interest</center>

| Variable Name | Mean | Median | Minimum | Maximum | Number of Observations |
|:-:|:-:|:-:|:-:|:-:|:-:|
|Player Age (years) | 21.14 | 19 | 9 | 58 | 196 |
|Played Hours (hours) | 5.85 | 0.1 | 0 | 223.1 | 196 |
|Elapsed Session Time (minutes) | 50.86 | 30 | 3 | 259 | 1535 |
|Start Date (Month Day)| June 24 | June 24 | April 06 | Sept 26 | 1535 |
|End Date (Month Day)| June 24 | June 23 | April 06 | Sept 26 | 1535 |

* describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
your report should include code which:
    * loads data 
    * wrangles and cleans the data to the format necessary for the planned analysis
    * performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
    * creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
    * performs the data analysis
    * creates a visualization of the analysis
       
* note: all figures should have a figure number and a legend
### Discussion:
* summarize what you found
* discuss whether this is what you expected to find
* discuss what impact could such findings have
* discuss what future questions could this lead to
### References
* You may include references if necessary, as long as they all have a consistent citation style.

In [ ]:
### Run this cell before continuing.
library(tidyverse)
library(tidymodels)
library(tidyclust)
library(repr)
library(GGally)
library(cowplot)
options(repr.matrix.max.rows = 6)

#### Read the Datasets from URLs

In [ ]:
players_data <- read_csv("https://raw.githubusercontent.com/wojpc/dsci100-project-008-group09/refs/heads/main/Data/players.csv")
sessions_data <- read_csv("https://raw.githubusercontent.com/wojpc/dsci100-project-008-group09/refs/heads/main/Data/sessions.csv")

In [ ]:
players_data
sessions_data

#### Left Join Data on Hashed Email and Wrangle Time
A new column called session_time is added to show elapsed time of session and times are converted from strings to more usable data. Sessions data is now combined with player information. original_start_time and original_end_time were useless so I removed them to make the table tidier and less redundant. 

In [ ]:
sessions_players_joined <- sessions_data |>
  left_join(players_data, by = "hashedEmail")

sessions_players_elapsed <- sessions_players_joined |>
    mutate(end_time = as.POSIXct(end_time, format = "%d/%m/%Y %H:%M"),
           start_time =  as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")) |>
    mutate(session_time_elapsed = as.numeric(end_time - start_time)) |>
    select(-original_start_time, -original_end_time)

sessions_players_elapsed

#### Preliminary Visualizations

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)

# Scatterplot of age and total hours played
players_data1 <- filter(players_data, played_hours > 0)
age_vs_hours <- ggplot(players_data1, aes(x = Age, y = played_hours)) +
        geom_point() +
        labs(x = "Age (yrs)",
             y = "Total Hours Played (hours)") +
        scale_y_log10() + theme_bw()

# Scatterplot of age and session time
sessions_players_elapsed1 <- filter(sessions_players_elapsed, session_time_elapsed > 0)
age_vs_sesstime <- ggplot(sessions_players_elapsed1, aes(x = Age, y = session_time_elapsed)) +
        geom_point() +
        labs(x = "Age (yrs)",
             y = "Total Session Time (minutes)") + theme_bw()

# Bar graph of experience and total hours played with gender as fill
players_data_col <- select(players_data, experience, played_hours, gender)
experience_playtime <- ggplot(players_data_col, aes(x = experience, 
                                                y = played_hours,
                                                color = gender,
                                                fill = gender)) +
        geom_col() +
        labs(x = "Experience",
             y = "Total Hours Played",
             fill = "Gender") +
        scale_fill_brewer(palette = "Set1") +
        scale_color_brewer(palette = "Set1") +
        guides(color = "none") + theme_bw()

# ggpairs plot of quantitative variables
player_pairs <- sessions_players_elapsed |> select(played_hours, Age, session_time_elapsed) |>
    ggpairs(aes(alpha = 0.05)) +
    theme(text = element_text(size = 20)) + theme_bw() 

# Isolate the times when playing starts and ignore dates
starting_times <- sessions_players_elapsed |>
    mutate(start_times = as.POSIXct(format(start_time, "%H:%M"),
      format = "%H:%M", tz = "UTC"),
          feature_sin = sin(2 * pi * hour(start_times) / 24),
feature_cos = cos(2 * pi * hour(start_times) / 24))

# Histogram of session start times
session_time_plot2 <- starting_times |> 
    ggplot(aes(x = start_times)) +
    geom_histogram() +
    labs(x = "Session Start Time (HH:MM)",
        y = "Count",
        title = "Histogram 1.5. Distribution of Starting Times for Sessions") +
    scale_x_datetime(date_labels = "%H:%M",
                    date_breaks = "1 hour") +
    theme_bw() +
    theme(text = element_text(size = 15),
         plot.title = element_text(hjust = 0.5),
         axis.text.x = element_text(angle = -45,
                                  vjust = 0.1))

# Scatterplot of start times and session lengths
start_vs_sesslen <- ggplot(starting_times, aes(x = start_times, y = session_time_elapsed)) +
        geom_point() +
        labs(x = "Session Start Time (HH:MM)",
             y = "Sess time (minutes)") +
        scale_x_datetime(date_labels = "%H:%M",
                         date_breaks = "1 hour") + theme_bw() +
theme(text = element_text(size = 15),
         plot.title = element_text(hjust = 0.5),
         axis.text.x = element_text(angle = -45,
                                  vjust = 0.1))


session_time_plot2
start_vs_sesslen
experience_playtime
age_vs_hours
age_vs_sesstime
player_pairs

#### kNN Means Clustering for Total Hours Played

In [ ]:
#Set seed for consistent reproduction 
set.seed(888)

#Plot size
options(repr.plot.width = 12, repr.plot.height = 5)

## Tune for best k##
total_played_clustdata <- players_data |> 
    select(Age, played_hours) |> 
    filter(played_hours > 0) |>
    filter(!is.na(Age))

total_played_recipe <- recipe(~ ., total_played_clustdata) |>
    step_log(played_hours, base = 10) |>
    step_scale(all_predictors(), na_rm = TRUE) |>
    step_center(all_predictors(), na_rm = TRUE)

ks <- tibble(num_clusters = 1:10) 

player_spec_nstart <- k_means(num_clusters = tune()) |>
    set_engine("stats", nstart = 100)

elbow_stats <- workflow() |>
    add_recipe(total_played_recipe) |>
    add_model(player_spec_nstart) |>
    tune_cluster(resamples = apparent(total_played_clustdata), grid = ks) |>
    collect_metrics() |>
    filter(.metric == "sse_within_total") |>
    mutate(total_WSSD = mean) |>
    select(num_clusters, total_WSSD)
    
elbow_plot <- elbow_stats |>
    ggplot(aes(x = num_clusters, y = total_WSSD)) +
    geom_point() +
    geom_line() +
    labs(x = "K",
        y = "Total within-cluster sum of squares") +
    theme(text = element_text(size = 15)) +
    scale_x_continuous(breaks = 1:10)

## Clustering with Optimal K ##
total_played_spec <- k_means(num_clusters = 3) |>
    set_engine("stats")

total_played_clustering <- workflow() |>
    add_recipe(total_played_recipe) |>
    add_model(total_played_spec) |>
    fit(data = total_played_clustdata)

clustered_players <- augment(total_played_clustering, total_played_clustdata)

clustering_plot <- clustered_players |>
    ggplot(aes(x = Age, y = played_hours, color = .pred_cluster)) +
    geom_point() +
    labs(x = "Age",
        y = "Hours Played",
        color = "Cluster") +
    scale_y_log10()

plot_grid(elbow_plot + theme_bw(), clustering_plot + theme_bw(), align = "hv")

#### kNN Means Clustering for Session Times

In [ ]:
#Set seed for consistent reproduction 
set.seed(888)

#Plot size
options(repr.plot.width = 12, repr.plot.height = 5)

## Tune for best k##
sess_clustdata <- sessions_players_elapsed |> 
    select(Age, session_time_elapsed) |>
     filter(!is.na(Age), !is.na(session_time_elapsed))

sess_recipe <- recipe(~ ., sess_clustdata) |>
    step_scale(all_predictors(), na_rm = TRUE) |>
    step_center(all_predictors(), na_rm = TRUE)

ks <- tibble(num_clusters = 1:10) 

sess_spec_nstart <- k_means(num_clusters = tune()) |>
    set_engine("stats", nstart = 100)

elbow_stats_sess <- workflow() |>
    add_recipe(sess_recipe) |>
    add_model(sess_spec_nstart) |>
    tune_cluster(resamples = apparent(sess_clustdata), grid = ks) |>
    collect_metrics() |>
    filter(.metric == "sse_within_total") |>
    mutate(total_WSSD = mean) |>
    select(num_clusters, total_WSSD)
    
elbow_plot_sess <- elbow_stats_sess |>
    ggplot(aes(x = num_clusters, y = total_WSSD)) +
    geom_point() +
    geom_line() +
    labs(x = "K",
        y = "Total within-cluster sum of squares") +
    theme(text = element_text(size = 15)) +
    scale_x_continuous(breaks = 1:10)

## Clustering with Optimal K ##
sess_spec <- k_means(num_clusters = 3) |>
    set_engine("stats")

sess_clustering <- workflow() |>
    add_recipe(sess_recipe) |>
    add_model(sess_spec) |>
    fit(data = sess_clustdata)

clustered_sess <- augment(sess_clustering, sess_clustdata)

clustering_plot_sess <- clustered_sess |>
    ggplot(aes(x = Age, y = session_time_elapsed, color = .pred_cluster)) +
    geom_point() +
    labs(x = "Age",
        y = "Session Time (minutes)",
        color = "Cluster")


plot_grid(elbow_plot_sess + theme_bw(), clustering_plot_sess + theme_bw())

#### Clustering for Age, Session Time and Total Play Time

In [ ]:
#Set seed for consistent reproduction 
set.seed(888)

#Plot size
options(repr.plot.width = 8, repr.plot.height = 8)

## Tune for best k ##
avg_sesstime <- sessions_players_elapsed |>
    select(name, Age, session_time_elapsed, played_hours) |>
    filter(!is.na(Age), 
           !is.na(session_time_elapsed), 
           !is.na(played_hours),
           !is.nan(played_hours)) |>
    filter(played_hours > 0) |>
    group_by(name) |>
    summarize(avg_session_mins = mean(session_time_elapsed))

ast_clustdata <- left_join(players_data, avg_sesstime, by = "name") |> 
    filter(!is.na(avg_session_mins)) |> 
    mutate(played_mins = played_hours * 60) |>
    select(Age, played_mins, avg_session_mins)

#ast_clustdata

ast_recipe <- recipe(~ ., ast_clustdata) |>
    step_scale(all_predictors(), na_rm = TRUE) |>
    step_center(all_predictors(), na_rm = TRUE)

ks <- tibble(num_clusters = 1:10) 

ast_spec_nstart <- k_means(num_clusters = tune()) |>
    set_engine("stats", nstart = 100)

elbow_stats_ast <- workflow() |>
    add_recipe(ast_recipe) |>
    add_model(ast_spec_nstart) |>
    tune_cluster(resamples = apparent(ast_clustdata), grid = ks) |>
    collect_metrics() |>
    filter(.metric == "sse_within_total") |>
    mutate(total_WSSD = mean) |>
    select(num_clusters, total_WSSD)
    
elbow_plot_ast <- elbow_stats_ast |>
    ggplot(aes(x = num_clusters, y = total_WSSD)) +
    geom_point() +
    geom_line() +
    labs(x = "K",
        y = "Total within-cluster sum of squares") +
    theme(text = element_text(size = 15)) +
    scale_x_continuous(breaks = 1:10) + theme_bw()

## Clustering with Optimal K ##
ast_spec <- k_means(num_clusters = 3) |>
    set_engine("stats")

ast_clustering <- workflow() |>
    add_recipe(ast_recipe) |>
    add_model(ast_spec) |>
    fit(data = ast_clustdata)

clustered_sess <- augment(ast_clustering, ast_clustdata)

clustered_sess_density <- clustered_sess  |>
    pivot_longer(cols = -.pred_cluster, names_to = 'category', values_to = 'value')  |> 
    ggplot(aes(value, fill = .pred_cluster)) +
        geom_density(alpha = 0.4, colour = 'white') +
        facet_wrap(facets = vars(category), scales = 'free') +
        theme_minimal() +
        theme(text = element_text(size = 20))

elbow_plot_ast

options(repr.plot.width = 12, repr.plot.height = 8)
clustered_sess_density

#### Bootstrapping and Inference for the Mean with 99% Confidence Interval

In [ ]:
#Set seed for consistent reproduction 
set.seed(888)
options(repr.plot.width = 8, repr.plot.height = 5)

## Age
age_mean <- mean(players_data$Age, na.rm = TRUE)

ages <- players_data |>
        filter(!is.na(Age))|>
        select(Age)

samples_age <- rep_sample_n(ages, size = 196, reps = 1500, replace = TRUE) |>
    group_by(replicate) |>
    summarize(mean_age = mean(Age))

age_mean

age_bootplot <- ggplot(samples_age, aes(x = mean_age)) +
    geom_histogram(binwidth = 1)

samples_age |> 
    select(mean_age) |> 
    pull() |>
    quantile(c(0.005, 0.995))

## Hours Played
played_mean <- mean(players_data$played_hours, na.rm = TRUE)

played_data <- players_data |>
        filter(!is.na(played_hours))|>
        select(played_hours)

samples_played <- rep_sample_n(played_data, size = 196, reps = 1500, replace = TRUE) |>
    group_by(replicate) |>
    summarize(mean_played_hours = mean(played_hours))

played_mean

played_bootplot <- ggplot(samples_played, aes(x = mean_played_hours)) +
    geom_histogram(binwidth = 1)

samples_played |> 
    select(mean_played_hours) |> 
    pull() |>
    quantile(c(0.005, 0.995))

## Session Time (Minutes)
sess_mean <- mean(sessions_players_elapsed$session_time_elapsed, na.rm = TRUE)

sess_data <- sessions_players_elapsed |>
        filter(!is.na(session_time_elapsed))|>
        select(session_time_elapsed)

samples_sess <- rep_sample_n(sess_data, size = 1533, reps = 1500, replace = TRUE) |>
    group_by(replicate) |>
    summarize(mean_sess_mins = mean(session_time_elapsed))

sess_mean

sess_bootplot <- ggplot(samples_sess, aes(x = mean_sess_mins)) +
    geom_histogram(binwidth = 1)

samples_sess |> 
    select(mean_sess_mins) |> 
    pull() |>
    quantile(c(0.005, 0.995))

plot_grid(age_bootplot, played_bootplot, sess_bootplot, ncol = 3)

#### kNN Classification for Experience

In [ ]:
#Set seed for consistent reproduction 
set.seed(888)
options(repr.plot.width = 5, repr.plot.height = 5)

players_data_factored <- players_data |>
                         mutate(experience = as_factor(experience)) |>
                         select(experience, Age, played_hours) |>
                         filter(!is.na(experience), !is.na(Age), !is.na(played_hours))

players_split <- initial_split(players_data_factored, prop = 4/5, strata = experience)
players_training <- training(players_split)
players_testing <- testing(players_split)

knn_spec_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

players_recipe <- recipe(experience ~ Age + played_hours, data = players_training) |>
                        step_scale(all_predictors()) |>
                        step_center(all_predictors())

players_vfold <- vfold_cv(players_training, v = 3, repeats = 3, strata = experience)

knn_results <- workflow() |>
                 add_recipe(players_recipe) |>
                 add_model(knn_spec_tune) |>
                 tune_grid(resamples = players_vfold, grid = tibble(neighbors = seq(1, 11, 1))) |>
                 collect_metrics()

accuracies <- knn_results |>
                 filter(.metric == 'accuracy')

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
                  geom_point() +
                  geom_line() +
                  labs(x = 'Neighbors', y = 'Accuracy Estimate') +
                  theme(text = element_text(size = 20)) +
                  scale_x_continuous(breaks = seq(1, 12, 1))
cross_val_plot


knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 4) |>
            set_engine("kknn") |>
            set_mode("classification")

knn_fit <- workflow() |>
           add_recipe(players_recipe) |>
           add_model(knn_spec) |>
           fit(data = players_training)

player_predictions <- predict(knn_fit, players_testing) |>
                        bind_cols(players_testing)

player_metrics <- player_predictions |> metrics(truth = experience, estimate = .pred_class)

player_conf_mat <- player_predictions |>
                        conf_mat(truth = experience, estimate = .pred_class) 
player_metrics
player_conf_mat

#### Linear Regression for Total Played Minutes Based on Age and Average Session Time

In [ ]:
# Linear regression
playersess_split <- initial_split(ast_clustdata , prop = 3/4, strata = played_mins)
training_players <- training(playersess_split)
testing_players <- testing(playersess_split)

lm_spec <- linear_reg(mode = "regression", 
                      engine = "lm")

lm_recipe <- recipe(played_mins ~ Age + avg_session_mins, data = training_players)

lm_fit <- workflow() |>
      add_recipe(lm_recipe) |>
      add_model(lm_spec) |>
      fit(training_players)
lm_fit

lm_rmse <- lm_fit |>
        predict(training_players) |>
        bind_cols(training_players) |>
        metrics(truth = played_mins, estimate = .pred) |>
        filter(.metric == "rmse") |>
        select(.estimate) |>
        pull()
lm_rmse

player_fit_testing <- workflow() |>
    add_recipe(lm_recipe) |>
    add_model(lm_spec) |>
    fit(testing_players)

lm_rmspe <- player_fit_testing |>
        predict(testing_players) |>
        bind_cols(testing_players) |>
        metrics(truth = played_mins, estimate = .pred) |>
        filter(.metric == "rmse") |>
        select(.estimate) |>
        pull()

lm_rmspe

#### kNN Regression for Total Played Minutes Based on Age and Average Session Time

In [ ]:
knnreg_spec <- nearest_neighbor(weight_func = 'rectangular', neighbors = tune()) |> 
      set_engine('kknn') |>
      set_mode('regression') 

knnreg_recipe <- recipe(played_mins ~ Age + avg_session_mins, data = training_players) |>
      step_scale(all_predictors()) |>
      step_center(all_predictors()) 

players_reg_vfold <- vfold_cv(training_players, 5,, strata = played_mins)

players_reg_workflow <- workflow() |>
    add_recipe(knnreg_recipe) |>
    add_model(knnreg_spec)

neighbors <- seq(1,81,10)
gridvals <- tibble(neighbors)
players_results <- players_reg_workflow |>
  tune_grid(resamples = players_reg_vfold, grid = gridvals) |>
  collect_metrics()

players_min <- players_results |>
   filter(.metric == 'rmse') |>
   slice_min(std_err)

players_min
neighbors
players_results